<a href="https://colab.research.google.com/github/shemaaali/DS-Unit-4-Sprint-1-NLP/blob/main/module4-topic-modeling/LS_DS_414_Topic_Modeling_Assignment/SA4_LS_DS_414_Topic_Modeling_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=aee9b7748a74659c9ecf2b71561efc5588367be0482593c365e254b28f26027c
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [9]:
!wget https://github.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/blob/main/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip?raw=true -O datafiniti.zip
!unzip datafiniti.zip

--2020-10-01 17:07:57--  https://github.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/blob/main/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip?raw=true
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/raw/main/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip [following]
--2020-10-01 17:07:57--  https://github.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/raw/main/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip [following

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

In [4]:
# creates gensim tokenizer
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [10]:
# import amazon dataset
amzn = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [11]:
# create column(tokenized data)
amzn['tokens'] = amzn['reviews.text'].apply(lambda x: tokenize(x))

In [12]:
# create dataframe with those columns
df = amzn[['primaryCategories', 'brand', 'name', 'tokens', 'reviews.rating', ]].copy()
df.head()

,primaryCategories,brand,name,tokens,reviews.rating
0,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[order, item, bad, quality, missing, backup, s...",3
1,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[bulk, expensive, way, products, like]",4
2,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[duracell, price, happy]",5
3,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[work, brand, batteries, better, price]",5
4,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[batteries, long, lasting, price, great]",5


In [13]:
# creating corpora
id2word = corpora.Dictionary(df['tokens'])

# filter all extreme values
id2word.filter_extremes(no_below=10, no_above=0.80)

# checks length of id2word
len(id2word)

2359

In [14]:
# creates corpus for each row
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [15]:
# setting up lda model
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=8,
                   passes=10,
                   workers=4
                  )

In [16]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.091487 -0.224205       1        1  22.462371
3     -0.187605  0.149518       2        1  17.845109
6     -0.131832 -0.004068       3        1  13.973109
2     -0.082416  0.085313       4        1  13.092728
0     -0.136524 -0.142548       5        1   9.771266
1      0.281989  0.108290       6        1   8.622443
5     -0.031691  0.001694       7        1   7.702809
7      0.196592  0.026007       8        1   6.530164, topic_info=           Term         Freq        Total Category  logprob  loglift
20        great  9131.000000  9131.000000  Default  30.0000  30.0000
33         good  5728.000000  5728.000000  Default  29.0000  29.0000
17    batteries  7370.000000  7370.000000  Default  28.0000  28.0000
16        price  4998.000000  4998.000000  Default  27.0000  27.0000
1831     tablet  7101.000000  7101.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
102        need   130.189774   946.100234   Topic8  -5.0108   0.7454
97          got   135.512873  1272.186641   Topic8  -4.9707   0.4893
27       bought   155.658852  3160.327264   Topic8  -4.8321  -0.2820
172        nice   127.053116  1223.786894   Topic8  -5.0352   0.4636
90          use   145.605517  4504.607392   Topic8  -4.8989  -0.7032

[575 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
55        1  0.998499       aa
56        1  0.997818      aaa
2005      3  0.024625  adapter
2005      5  0.960387  adapter
1662      2  0.005473      ads
...     ...       ...      ...
101       1  0.043013     year
101       2  0.930594     year
101       4  0.009300     year
101       5  0.016856     year
1463      2  0.996521       yr

[1545 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 7, 3, 1, 2, 6, 8])

In [17]:
# show topics with regex
word_list = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

# create topic list
topics = []
for words in word_list:
    topic = ' '.join(words)
    topics.append(topic)

In [18]:
lda_values = [lda[x] for x in corpus]

In [19]:
# function for formating data 
def update(doc):
        d_dist = {k:0 for k in range(0,8)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in lda_values]

In [24]:
df1 = pd.DataFrame.from_records(new_distro)
df1.columns = topics
df1.head()

,bought charge music amazon work alexa kindle use home charging,great price product good value deal works recommend buy work,tablet amazon apps great ipad movies love games screen google,tablet loves old year kids bought love easy use games,batteries long work amazon brand battery brands price time buy,use tablet easy battery life amazon like good great kids,kindle screen love easy reading read books size use like,good tablet price far best buy quality expected better works
0,0.012525,0.144827,0.012519,0.012502,0.584324,0.208279,0.012510,0.012514
1,0.020848,0.020843,0.020850,0.020843,0.854037,0.020850,0.020856,0.020873
2,0.031286,0.443196,0.031262,0.031271,0.369064,0.031365,0.031289,0.031268
3,0.020838,0.020922,0.020844,0.020835,0.853997,0.020836,0.020846,0.020883
4,0.020834,0.343661,0.020837,0.020836,0.531318,0.020836,0.020836,0.020842


In [25]:
df2 = pd.DataFrame(new_distro)
df2.columns = topics
df2.head()

,bought charge music amazon work alexa kindle use home charging,great price product good value deal works recommend buy work,tablet amazon apps great ipad movies love games screen google,tablet loves old year kids bought love easy use games,batteries long work amazon brand battery brands price time buy,use tablet easy battery life amazon like good great kids,kindle screen love easy reading read books size use like,good tablet price far best buy quality expected better works
0,0.012525,0.144827,0.012519,0.012502,0.584324,0.208279,0.012510,0.012514
1,0.020848,0.020843,0.020850,0.020843,0.854037,0.020850,0.020856,0.020873
2,0.031286,0.443196,0.031262,0.031271,0.369064,0.031365,0.031289,0.031268
3,0.020838,0.020922,0.020844,0.020835,0.853997,0.020836,0.020846,0.020883
4,0.020834,0.343661,0.020837,0.020836,0.531318,0.020836,0.020836,0.020842


In [27]:
# concat df1 with category dataframe
finalDF = pd.concat([df['primaryCategories'], df1], axis=1)

In [28]:
# look at topics
# used for different category reviews

finalDF.groupby('primaryCategories').mean()

,bought charge music amazon work alexa kindle use home charging,great price product good value deal works recommend buy work,tablet amazon apps great ipad movies love games screen google,tablet loves old year kids bought love easy use games,batteries long work amazon brand battery brands price time buy,use tablet easy battery life amazon like good great kids,kindle screen love easy reading read books size use like,good tablet price far best buy quality expected better works
primaryCategories,,,,,,,,
Animals & Pet Supplies,0.037757,0.238941,0.015337,0.110512,0.318391,0.015332,0.238765,0.015338
Electronics,0.100465,0.093340,0.152504,0.268323,0.024496,0.092980,0.175616,0.079993
"Electronics,Furniture",0.484145,0.000000,0.167038,0.191658,0.000000,0.110778,0.000000,0.000000
"Electronics,Media",0.068208,0.054447,0.021464,0.040311,0.028999,0.073909,0.673575,0.023729
Health & Beauty,0.043969,0.230722,0.033057,0.034870,0.441477,0.050094,0.033911,0.124611
Home & Garden,0.041687,0.041682,0.041743,0.299946,0.200024,0.291503,0.041748,0.041668
Office Supplies,0.028120,0.363321,0.204753,0.028128,0.028120,0.045409,0.061554,0.240597
"Office Supplies,Electronics",0.071215,0.091532,0.075940,0.107051,0.025626,0.082936,0.468726,0.065465
"Toys & Games,Electronics",0.070211,0.065055,0.107319,0.558094,0.022137,0.071526,0.039712,0.052370


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling